In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from pdfminer.high_level import extract_text
import os

>We'll load the information

In [889]:
with open('dictionary_speeches_2.pkl', 'rb') as fl:
    information = pkl.load(fl)

In [891]:
len(information.keys())

417

>Seems like we loaded everthing

# Let's load load the PDFs

In [892]:
lt_fl = os.listdir('duterte_pdf2')
transcript_dict = {}

for fl in lt_fl:
    with open(f'duterte_pdf2\{fl}', 'rb'):
        date = fl[:-4]
        transcript_dict[date] = extract_text(f'duterte_pdf2\\{fl}')
transcript_dict.keys()

dict_keys(['April 02, 2018', 'April 03, 2018', 'April 05, 2018', 'April 05, 2018_2', 'April 09, 2018', 'April 10, 2018', 'April 10, 2018_2', 'April 10, 2018_3', 'April 12, 2018', 'April 13, 2018', 'April 15, 2018', 'April 17, 2018', 'April 17, 2018_2', 'April 18, 2018', 'April 19, 2018', 'April 21, 2018', 'April 26, 2018', 'April 28, 2018', 'April 28, 2018_2', 'August 17, 2017', 'August 28, 2017_2', 'August 29, 2017', 'February 01, 2018', 'February 06, 2018', 'February 06, 2018_2', 'February 07, 2018', 'February 09, 2018', 'February 09, 2018_2', 'February 12, 2018', 'February 12, 2018_2', 'February 13, 2018', 'February 13, 2018_2', 'February 13, 2018_3', 'February 19, 2018', 'February 20, 2018', 'February 21, 2018', 'February 22, 2018', 'February 22, 2018_2', 'February 26, 2018', 'February 28, 2018', 'February 28, 2018_2', 'July 20, 2017', 'July 24, 2017', 'July 25, 2017', 'March 01, 2018', 'March 06, 2018', 'March 07, 2018', 'March 07, 2018_2', 'March 07, 2018_3', 'March 11, 2018', 'M

In [893]:
len(transcript_dict.keys())

99

>So we loaded everything

>Like I did before I'm going to check if they contain the pattern I discussed in (put section number)

In [2]:
import re
regex = re.compile('\\[(.*?)\]')
for dt, trans in zip(transcript_dict.keys(), transcript_dict.values()):
        if not re.search(regex, trans.replace('\n', '')):
            print(dt)

NameError: name 'transcript_dict' is not defined

>So all of them has the pattern

>Let's check which pdf contains dialogue of another person (same procedure we did at put section number)

In [899]:
lt_names= ['PRESIDENT RODRIGO DUTERTE:', 'PRESIDENT DUTERTE:', 'PRESIDENT RODRIGO ROA DUTERTE:']

In [900]:
lt_contain_convo = []
for dt, trans in zip(transcript_dict.keys(), transcript_dict.values()):
    if any(nm in trans for nm in lt_names):
        lt_contain_convo.append(dt)
lt_contain_convo

['April 09, 2018',
 'April 13, 2018',
 'February 09, 2018_2',
 'February 22, 2018_2',
 'February 28, 2018',
 'May 16, 2018',
 'May 16, 2018_2',
 'May 16, 2018_3',
 'November 08, 2017',
 'November 09, 2017',
 'November 13, 2017',
 'November 14, 2017',
 'October 31, 2017',
 'September 03, 2017',
 'September 05, 2017',
 'September 09, 2017']

>These are the pdfs that contains dialogue of another person. I used <a href='https://pdf2docx.com/'>pdf2docx</a> to convert them to docx using  ro be processed later on.

>I'm just going to store them to another file

In [55]:
import shutil

for fl in lt_contain_convo:
    shutil.copy(f'duterte_pdf2\\{fl}.pdf', f'duterte_docx2\\{fl}.pdf')

>Now I'm going to scrape only Presiden Duterte's dialogue. Some pdf actually contain the location of the event so I'm going to scrape these also.

In [907]:
import docx

loc_trans_docx = {}

for fl in lt_contain_convo:
    
    encountered = False
    isDuterte = True
    trans = []
    
    doc = docx.Document(f'duterte_docx2\\{fl}.docx')
    
    for paragraph in doc.paragraphs:
        if encountered:
            for run in paragraph.runs:
                # if the text is bold we ask if it is a name of duterte or not
                 if run.bold:
                    if any(nm in paragraph.text for nm in lt_names):
                                isDuterte = True
                    else:
                        isDuterte = False
                        
        #we'll only include President Duterte's dialogue in the list
        if isDuterte and encountered:
            if paragraph.text:
                trans.append(paragraph.text)


        #if we encounter the pattern
        if re.search(regex, paragraph.text.replace('\n', '')) and not encountered:
            encountered = True
            #we'll extract the location
            pattern = paragraph.text
            separator = pattern.find('|')
            loc = pattern[1:separator]
            if 'Delivered at' in loc:
                loc = loc[len('Delivered at'):].strip()
            
    
    single_str = ' '.join(trans)
    
    #remove duterte's name
    for nm in lt_names:
        single_str = single_str.replace(nm, ' ')
        
    loc_trans_docx[fl] = {'location':loc, 'transcript': single_str}

In [908]:
len(loc_trans_docx)

16

>Now we'll scrape dialogue of President Duterte from speeches that has only his dialogue.

In [909]:
loc_trans_pdf = {}
for dt, transcript in zip(transcript_dict.keys(), transcript_dict.values()):
    if dt not in loc_trans_docx.keys():
        cleaned = transcript.replace('\n', ' ')
        try:
            span = re.search(regex, cleaned).span()
        except:
            print(dt)

        #we'll extract the location
        pattern = cleaned[span[0]:span[1]]
        separator = pattern.find('|')
        loc = pattern[1:separator]
        if 'Delivered at' in loc:
            loc = loc[len('Delivered at'):].strip()

        transcript = cleaned[span[1]:].strip()

        loc_trans_pdf[dt] = {'location':loc, 'date': dt, 'transcript':transcript,}

In [910]:
len(loc_trans_pdf)

83

>Now we're going to merge the two dictionaries.

In [911]:
loc_trans_pdf.update(loc_trans_docx)
len(loc_trans_pdf)

99

>Let's update the *transcript* of these speeches in the dictionary *information*. Let's check first if all keys from these *loc_trans_df* exist in *information*.

In [912]:
for key in loc_trans_pdf.keys():
    if key not in information.keys():
        print(key)

>Let's check if the *transcripts* of the pdf files in *information* are empty.

In [913]:
for key in loc_trans_pdf.keys():
    if key in information.keys():
        if type(information[key]['transcript']) != float:
            print(information[key])

>Looks like they are all empty so let's add the transcripts for those speeches with empty *transcripts* in *information*.

In [914]:
for key in loc_trans_pdf.keys():
    information[key]['transcript'] = loc_trans_pdf[key]['transcript']
    information[key]['location'] = loc_trans_pdf[key]['location']    

In [915]:
len(information.keys())

417

>Now that we have everything let's turn it into dataframe

In [916]:
df_cur = pd.DataFrame(information).T
df_cur

,title,date,location,event,transcript,url
"June 30, 2016",President Rodrigo Roa Duterte’s Speech during ...,"June 30, 2016","Rizal Ceremonial Hall, Malacañang Palace",NaN,"President Fidel Ramos, sir, salamat po sa tulo...",http://pcoo.gov.ph//june-30-2016-president-rod...
"July 31, 2016",Remarks of President Rodrigo Roa Duterte durin...,"July 31, 2016","PSG Grandstand, Malacañang Park, Manila",NaN,Let me begin by addressing myself to you throu...,http://pcoo.gov.ph//july-31-2016-remarks-of-pr...
"July 29, 2016",Speech of President Rodrigo Roa Duterte during...,"July 29, 2016","Purok 35, Phase 3, Acasia St., Bgy. 76-A, Dava...",NaN,Maayong gabi-I kaninyong tanan. (audience resp...,http://pcoo.gov.ph//july-29-2016-speech-of-pre...
"July 29, 2016_2",Speech of President Rodrigo Roa Duterte during...,"July 29, 2016","60th IB headquarters, Asuncion, Daval del Norte",NaN,I’d like to address myself to the Filipino peo...,http://pcoo.gov.ph//july-29-2016-speech-of-pre...
"July 28, 2016",Speech of President Rodrigo Roa Duterte during...,"July 28, 2016","Lucena City, Quezon Province",NaN,Secretary Delfin Lorenzana; Secretary Arthur T...,http://pcoo.gov.ph//july-28-2016-speech-of-pre...
...,...,...,...,...,...,...
"May 08, 2018",Speech of President Rodrigo Roa Duterte during...,"May 08, 2018","the Rizal Hall, Malacañan Palace",NaN,"Maybe after my prepared speech, this is just a...",http://pcoo.gov.ph/wp-content/uploads/2018/05/...
"May 05, 2018",Speech of President Rodrigo Roa Duterte during...,"May 05, 2018","the Isla Ballroom, EDSA Shangri-La, Mandaluyon...",NaN,Thank you. Kindly sit down. I’d forego wit...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...
"May 04, 2018",Speech of President Rodrigo Roa Duterte during...,"May 04, 2018","the SMX Convention Center, Davao City",NaN,Salamat po. Thank you for your courtesy. You m...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...
"May 02, 2018",Speech of President Rodrigo Roa Duterte during...,"May 02, 2018","Liwasang Alfaro G. Aguirre, Mulanay, Quezon",NaN,Kindly sit down. Thank you for your courtesy. ...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...


>Let's load the dataframe from previous scraping

In [917]:
df_prev = pd.read_csv('Duterte_Speech2.csv')

In [918]:
df_prev

,date,title,url,event,location,transcript
0,"May 26, 2020",Excerpts from Speech of President Rodrigo Roa ...,https://pcoo.gov.ph/presidential-speech/excerp...,Meeting with Philippine Army (PA) and Philippi...,"Malago Clubhouse, Malacañang Park, Manila",So ako pati si Bong during my mayorship d...
1,"May 25, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: I remember dist...
2,"May 22, 2020",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Commencement Exercsies of the Philippine Milit...,"Malago Clubhouse, Malacañang Park, Manila","Kindly sit down. [May upuan sila? Okay.], Defe..."
3,"May 19, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: Good evening my...
4,"May 12, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,"PRESIDENT DUTERTE: Sir, one question. Itong op..."
...,...,...,...,...,...,...
311,"April 3, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Inauguration of Lisap Bridge Project,"Bongabong, Oriental Mindoro",Nagkahiyaan pa. Sige na. Kaya ‘pag mag-usap [...
312,"April 2, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Distribution of Certificate of Land Ownership ...,"Provincial Capitol Gymnasium, Isulan, Sultan K...",Kindly sit down. Salamat po sa courtesy n...
313,"March 26, 2018",Speech Of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Handover of loose firearms and presentation of...,"Multi-purpose Gym, Capitol, Patikul Sulu",Thank you for your courtesy. Kindly sit d...
314,"March 23, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,16th founding Anniversary of the Supreme Triba...,"3rd Infantry Battalion Headquarters, Calinan, ...",Salamat. Panglingkod mo. Kanang... Una sa tan...


>Some speeches overlap between two dataframe, let's identify them

In [919]:
exist_both = []
for key in df_prev.date.tolist():
    if any(df_cur.date.str.contains(key).tolist()):
        print(key)
        exist_both.append(key)

May 19, 2018
May 16, 2018
May 15, 2018
May 11, 2018
April 28, 2018
April 26, 2018
April 21, 2018
April 19, 2018
April 18, 2018
April 17, 2018
April 15, 2018
April 13, 2018
April 12, 2018
April 10, 2018
March 23, 2018


>So these speeches exist in both dataframe. Since speeches in *df_prev* contains complete information than speeches in *df_cur* I'm gonna delete the duplicates in *df_cur*. While deleting tho I have to check which exact speech in *df_cur* does the speech corresponds to as there could be multiple speeches in that date.

In [920]:
import pickle as pkl
with open('df_prev.pkl', 'wb') as fl:
    pkl.dump(df_prev, fl)
    
with open('df_cur.pkl', 'wb') as fl:
    pkl.dump(df_cur, fl)

In [921]:
df_cur_2 = df_cur
df_prev_2 = df_prev

# if you want to load

In [927]:
import pickle as pkl
with open('df_prev.pkl', 'rb') as fl:
    df_prev = pkl.load(fl)
    
with open('df_cur.pkl', 'rb') as fl:
    df_cur = pkl.load(fl)

In [925]:
exist_both

['May 19, 2018',
 'May 16, 2018',
 'May 15, 2018',
 'May 11, 2018',
 'April 28, 2018',
 'April 26, 2018',
 'April 21, 2018',
 'April 19, 2018',
 'April 18, 2018',
 'April 17, 2018',
 'April 15, 2018',
 'April 13, 2018',
 'April 12, 2018',
 'April 10, 2018',
 'March 23, 2018']

In [933]:
df_prev[df_prev.date == 'May 19, 2018'].title.values

array(['Speech of President Roa Duterte during the Ceremonial Opening of Oil and Gas Production of the Alegria Oilfield Polyard-3 Well Site (with translation)'],
      dtype=object)

In [937]:
df_cur[df_cur.date.str.contains('May 19, 2018')].title.values[3]

'Speech of President Rodrigo Roa Duterte during the Ceremonial Opening of Oil and Gas Production of the Alegria Oilfield Polyard-3 Well Site (with translation)'

In [938]:
df_prev.loc[df_prev.date == 'May 19, 2018', 'title'] = 'Speech of President Rodrigo Roa Duterte during the Ceremonial Opening of Oil and Gas Production of the Alegria Oilfield Polyard-3 Well Site (with translation)'

>While running the block of code below I got an error because the two dataframe has date for May 19, 2018 but none in *df_cur* has the same title. Turns outs the one in *df_prev* lack the word Rodrigo for the full nname of President Duterte(see output of the two cells above). I have to a manually the on one in *df_prev*.

In [940]:
for dt in exist_both:
    temp_df = df_cur[df_cur.date.str.contains(dt)]
    if temp_df.shape[0] > 1:
        #check which speech in df_cur it correspond to
        actual = temp_df[temp_df.title == df_prev[df_prev.date == dt].title.values[0]]
        actual_date = actual.index.values
        #delete in df_cur
        df_cur.drop(index=actual_date, inplace=True)
        
        #change date in df_prev
        df_prev.loc[df_prev.date == dt, 'date'] = actual_date
    else:
        df_cur.drop(index=dt, inplace=True)
        


In [941]:
df_prev.date.tolist()

['May 26, 2020',
 'May 25, 2020',
 'May 22, 2020',
 'May 19, 2020',
 'May 12, 2020',
 'May 4, 2020',
 'April 27, 2020',
 'April 24, 2020',
 'April 16, 2020',
 'April 14, 2020',
 'April 13, 2020',
 'April 8, 2020',
 'April 6, 2020',
 'April 3, 2020',
 'April 1, 2020',
 'March 31, 2020',
 'March 24, 2020',
 'March 19, 2020',
 'March 16, 2020',
 'March 12, 2020',
 'March 11, 2020',
 'March 10, 2020',
 'March 9, 2020',
 'March 5, 2020',
 'March 3, 2020',
 'February 26, 2020',
 'February 25, 2020',
 'February 20, 2020',
 'February 15, 2020',
 'February 12, 2020',
 'February 11, 2020',
 'February 10, 2020',
 'February 6, 2020',
 'February 3, 2020',
 'January 29, 2020',
 'January 24, 2020',
 'January 21, 2020',
 'January 20, 2020',
 'January 19, 2020',
 'January 17, 2020',
 'January 16, 2020',
 'January 14, 2020',
 'January 13, 2020',
 'January 10, 2020',
 'January 8, 2020',
 'January 7, 2020',
 'January 6, 2020',
 'January 4, 2020',
 'December 30, 2019',
 'December 23, 2019',
 'December 20, 

>Before we merge this two let's revise the format for *df_cur* so that they have the same format to *df_prev*

In [942]:
df_cur.drop(columns='date', inplace=True)
df_cur.reset_index(inplace=True)
df_cur.rename(columns={'index':'date'}, inplace=True)
df_cur

,date,title,location,event,transcript,url
0,"June 30, 2016",President Rodrigo Roa Duterte’s Speech during ...,"Rizal Ceremonial Hall, Malacañang Palace",NaN,"President Fidel Ramos, sir, salamat po sa tulo...",http://pcoo.gov.ph//june-30-2016-president-rod...
1,"July 31, 2016",Remarks of President Rodrigo Roa Duterte durin...,"PSG Grandstand, Malacañang Park, Manila",NaN,Let me begin by addressing myself to you throu...,http://pcoo.gov.ph//july-31-2016-remarks-of-pr...
2,"July 29, 2016",Speech of President Rodrigo Roa Duterte during...,"Purok 35, Phase 3, Acasia St., Bgy. 76-A, Dava...",NaN,Maayong gabi-I kaninyong tanan. (audience resp...,http://pcoo.gov.ph//july-29-2016-speech-of-pre...
3,"July 29, 2016_2",Speech of President Rodrigo Roa Duterte during...,"60th IB headquarters, Asuncion, Daval del Norte",NaN,I’d like to address myself to the Filipino peo...,http://pcoo.gov.ph//july-29-2016-speech-of-pre...
4,"July 28, 2016",Speech of President Rodrigo Roa Duterte during...,"Lucena City, Quezon Province",NaN,Secretary Delfin Lorenzana; Secretary Arthur T...,http://pcoo.gov.ph//july-28-2016-speech-of-pre...
...,...,...,...,...,...,...
397,"May 08, 2018",Speech of President Rodrigo Roa Duterte during...,"the Rizal Hall, Malacañan Palace",NaN,"Maybe after my prepared speech, this is just a...",http://pcoo.gov.ph/wp-content/uploads/2018/05/...
398,"May 05, 2018",Speech of President Rodrigo Roa Duterte during...,"the Isla Ballroom, EDSA Shangri-La, Mandaluyon...",NaN,Thank you. Kindly sit down. I’d forego wit...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...
399,"May 04, 2018",Speech of President Rodrigo Roa Duterte during...,"the SMX Convention Center, Davao City",NaN,Salamat po. Thank you for your courtesy. You m...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...
400,"May 02, 2018",Speech of President Rodrigo Roa Duterte during...,"Liwasang Alfaro G. Aguirre, Mulanay, Quezon",NaN,Kindly sit down. Thank you for your courtesy. ...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...


In [943]:
df_prev

,date,title,url,event,location,transcript
0,"May 26, 2020",Excerpts from Speech of President Rodrigo Roa ...,https://pcoo.gov.ph/presidential-speech/excerp...,Meeting with Philippine Army (PA) and Philippi...,"Malago Clubhouse, Malacañang Park, Manila",So ako pati si Bong during my mayorship d...
1,"May 25, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: I remember dist...
2,"May 22, 2020",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Commencement Exercsies of the Philippine Milit...,"Malago Clubhouse, Malacañang Park, Manila","Kindly sit down. [May upuan sila? Okay.], Defe..."
3,"May 19, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: Good evening my...
4,"May 12, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,"PRESIDENT DUTERTE: Sir, one question. Itong op..."
...,...,...,...,...,...,...
311,"April 3, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Inauguration of Lisap Bridge Project,"Bongabong, Oriental Mindoro",Nagkahiyaan pa. Sige na. Kaya ‘pag mag-usap [...
312,"April 2, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Distribution of Certificate of Land Ownership ...,"Provincial Capitol Gymnasium, Isulan, Sultan K...",Kindly sit down. Salamat po sa courtesy n...
313,"March 26, 2018",Speech Of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Handover of loose firearms and presentation of...,"Multi-purpose Gym, Capitol, Patikul Sulu",Thank you for your courtesy. Kindly sit d...
314,"March 23, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,16th founding Anniversary of the Supreme Triba...,"3rd Infantry Battalion Headquarters, Calinan, ...",Salamat. Panglingkod mo. Kanang... Una sa tan...


>Now that we have same column names let's merge them

In [944]:
df_all = pd.merge(df_prev, df_cur, how='outer')
df_all

,date,title,url,event,location,transcript
0,"May 26, 2020",Excerpts from Speech of President Rodrigo Roa ...,https://pcoo.gov.ph/presidential-speech/excerp...,Meeting with Philippine Army (PA) and Philippi...,"Malago Clubhouse, Malacañang Park, Manila",So ako pati si Bong during my mayorship d...
1,"May 25, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: I remember dist...
2,"May 22, 2020",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Commencement Exercsies of the Philippine Milit...,"Malago Clubhouse, Malacañang Park, Manila","Kindly sit down. [May upuan sila? Okay.], Defe..."
3,"May 19, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: Good evening my...
4,"May 12, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,"PRESIDENT DUTERTE: Sir, one question. Itong op..."
...,...,...,...,...,...,...
713,"May 08, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the Rizal Hall, Malacañan Palace","Maybe after my prepared speech, this is just a..."
714,"May 05, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the Isla Ballroom, EDSA Shangri-La, Mandaluyon...",Thank you. Kindly sit down. I’d forego wit...
715,"May 04, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the SMX Convention Center, Davao City",Salamat po. Thank you for your courtesy. You m...
716,"May 02, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"Liwasang Alfaro G. Aguirre, Mulanay, Quezon",Kindly sit down. Thank you for your courtesy. ...


>Although the date are unique, let's see if there are duplicate transcripts.

In [945]:
df_all.transcript.nunique()

708

>So only 708 transcripts are unique, while all other are just duplicates. Let's view these transcripts.

In [946]:
grouped_transcript = df_all.groupby('transcript')
u_transcript = df_all.transcript.unique()

u_transcript_multiple = []
for trans in u_transcript:
    try:
        if grouped_transcript.get_group(trans).shape[0] > 1:
            u_transcript_multiple.append(trans)
    except:
        print(trans)
len(u_transcript_multiple)

nan


7

>There are total of 7 kinds transcripts that has duplicates. I'm gonna manually check these and see the reason for it and fix them. I found out that most of them are just duplicates while the elements in 8 and 9 are actually 

In [967]:
for i in range(7):
    print(f'Group-{i}\n{grouped_transcript.get_group(u_transcript_multiple[i]).url.tolist()}')
    print()

Group-0
['http://pcoo.gov.ph//aug-29-2016-speech-of-president-rodrigo-roa-duterte-during-the-18th-anniversary-of-volunteers-against-crime-and-corruption-vacc/', 'http://pcoo.gov.ph//aug-29-2016-speech-of-president-rodrigo-roa-duterte-during-the-18th-anniversary-of-volunteers-against-crime-and-corruption-vacc/']

Group-1
['http://pcoo.gov.ph//aug-29-2016-statement-of-president-rodrigo-roa-duterte-during-his-visit-to-wounded-in-action-soldiers-in-tacloban-city/', 'http://pcoo.gov.ph//aug-29-2016-statement-of-president-rodrigo-roa-duterte-during-his-visit-to-wounded-in-action-soldiers-in-tacloban-city/']

Group-2
['http://pcoo.gov.ph//oct-23-2016-president-rodrigo-roa-dutertes-speech-during-the-distribution-of-emergency-shelter-assistance-esa-and-relief-goods-to-typhoon-lawin-victims/', 'http://pcoo.gov.ph//oct-23-2016-president-rodrigo-roa-dutertes-speech-during-the-distribution-of-emergency-shelter-assistance-esa-and-relief-goods-to-typhoon-lawin-victims/', 'http://pcoo.gov.ph//oct-23-2

>As you can see they are basically repeated speech with the same source so I'm going to delete their duplicate/s.

In [984]:
df_all_temp = df_all.copy()

In [1013]:
for i in range(7):
    df_all.drop(index=grouped_transcript.get_group(u_transcript_multiple[i]).index[1:], inplace=True)
    grouped_transcript = df_all.groupby('transcript')

In [1008]:
df_all = df_all_temp.copy()

In [1025]:
print(f'Shape of_all:{df_all.shape}')
print(f'Number of unique transcript:{df_all.transcript.nunique()}')
print(f'Number of nan transcript:{df_all.transcript.isna().sum()}')

Shape of_all:(709, 6)
Number of unique transcript:708
Number of nan transcript:1


In [1023]:
df_all[df_all.transcript.isna()]

,date,title,url,event,location,transcript
300,"April 25, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Launching of the Maguindanao Balik Baril Program,"Buluan, Maguindanao",NaN


>There is a discrepancy between nunique and shape because for *nunique()* function a nan value is not regarded as unique value. This nan value is the pdf that is not available from the site. I have to drop it from the dataframe.

In [1029]:
df_all.drop(index=df_all[df_all.transcript.isna()].index, inplace=True)
df_all

,date,title,url,event,location,transcript
0,"May 26, 2020",Excerpts from Speech of President Rodrigo Roa ...,https://pcoo.gov.ph/presidential-speech/excerp...,Meeting with Philippine Army (PA) and Philippi...,"Malago Clubhouse, Malacañang Park, Manila",So ako pati si Bong during my mayorship d...
1,"May 25, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: I remember dist...
2,"May 22, 2020",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Commencement Exercsies of the Philippine Milit...,"Malago Clubhouse, Malacañang Park, Manila","Kindly sit down. [May upuan sila? Okay.], Defe..."
3,"May 19, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,PRESIDENT RODRIGO ROA DUTERTE: Good evening my...
4,"May 12, 2020",Talk to the People of President Rodrigo Roa Du...,https://pcoo.gov.ph/presidential-speech/talk-t...,On Coronavirus Disease 2019 (COVID-19),Malago Clubhouse in Malacañang,"PRESIDENT DUTERTE: Sir, one question. Itong op..."
...,...,...,...,...,...,...
713,"May 08, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the Rizal Hall, Malacañan Palace","Maybe after my prepared speech, this is just a..."
714,"May 05, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the Isla Ballroom, EDSA Shangri-La, Mandaluyon...",Thank you. Kindly sit down. I’d forego wit...
715,"May 04, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"the SMX Convention Center, Davao City",Salamat po. Thank you for your courtesy. You m...
716,"May 02, 2018",Speech of President Rodrigo Roa Duterte during...,http://pcoo.gov.ph/wp-content/uploads/2018/05/...,NaN,"Liwasang Alfaro G. Aguirre, Mulanay, Quezon",Kindly sit down. Thank you for your courtesy. ...


>We have a total of 708 unique speeches from President Duterte.

In [1033]:
import json
import pickle as pkl

with open('Duterte_Dialogue.pkl', 'wb') as fl:
    pkl.dump(df_all, fl)
    
df_all.to_json('Duterte_Dialogue.json')
df_all.to_csv('Duterte_Dialogue.csv', index=False)

In [1044]:
df_all[df_all.date == 'October 19, 2016'].transcript.values[0]

'OPENING STATEMENT:\nPRESIDENT DUTERTE: Presently we are the subject of the fury of a powerful typhoon cyclone Lawin and it is hitting the country by now.\nAs usual, we have made the preparations to mitigate damages, save lives, and we have deployed all available measures to lessen at least to put into action what would really be a life-saving measures for my fellowmen.\n There are three leading agencies there, the DSWD which is really the social workers; and the mitigating guys, the National Disaster Risk Reduction and Management Council. There are all there on the ground today and we can only pray that it would not really result in so many damage and hopefully God forbid no lives lost.\n I cannot give you the…I think it’s going on now and I’m just waiting for a final report. Just before coming down, I read something that it’s coming in. So I suppose that I cannot have the facts to tell you know but maybe later in the evening or earliest tomorrow.\n I’m here to answer your questions a

In [1022]:
df_all[df_all.transcript.isna()]

,date,title,url,event,location,transcript
300,"April 25, 2018",Speech of President Rodrigo Roa Duterte during...,https://pcoo.gov.ph/presidential-speech/speech...,Launching of the Maguindanao Balik Baril Program,"Buluan, Maguindanao",NaN


# TEST

In [700]:
import requests
from bs4 import BeautifulSoup

In [723]:
url = 'http://pcoo.gov.ph//may-13-2017-speech-of-president-rodrigo-roa-duterte-during-the-meeting-with-the-filipino-community-in-hong-kong-special-administrative-region/'
headers = {
    'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1 RuxitSynthetic/1.0 v1355348020 t3296535826494656701 smf=0'
}
response = requests.get(url, headers=headers)
html = BeautifulSoup(response.text, 'lxml')

# SOLUTION TO '\xa0'